# モデルの改良

前回の結果をみると、手書き文字の精度として低い。

参考までに前回の結果を提示する。

In [2]:
!python3 diffusion_model.py

 19%|███████▊                                  | 87/469 [02:35<11:21,  1.78s/it]
Traceback (most recent call last):
  File "/Users/yoshidajunichi/Documents/DeepLearning_study/study/deep-learning-from-scratch-5/step09/diffusion_model.py", line 202, in <module>
    noise_pred = model(x_noisy, t)
  File "/Users/yoshidajunichi/.pyenv/versions/anaconda3-2021.11/lib/python3.9/site-packages/torch/nn/modules/module.py", line 1511, in _wrapped_call_impl
    return self._call_impl(*args, **kwargs)
  File "/Users/yoshidajunichi/.pyenv/versions/anaconda3-2021.11/lib/python3.9/site-packages/torch/nn/modules/module.py", line 1520, in _call_impl
    return forward_call(*args, **kwargs)
  File "/Users/yoshidajunichi/Documents/DeepLearning_study/study/deep-learning-from-scratch-5/step09/diffusion_model.py", line 104, in forward
    x = torch.cat([x, x1], dim=1)
KeyboardInterrupt


In [143]:
print(model)

UNet(
  (down1): ConvBlock(
    (convs): Sequential(
      (0): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
      (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (5): ReLU()
    )
    (mlp): Sequential(
      (0): Linear(in_features=100, out_features=64, bias=True)
      (1): ReLU()
      (2): Linear(in_features=64, out_features=64, bias=True)
    )
  )
  (down2): ConvBlock(
    (convs): Sequential(
      (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
      (3): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (4): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_

In [53]:
import torch
from torch import nn
import math

class ConvBlock(nn.Module):
    def __init__(self, in_ch, out_ch, time_embed_dim):
        super().__init__()
        self.convs = nn.Sequential(
            nn.Conv2d(in_ch, out_ch, 3, padding=1),
            nn.BatchNorm2d(out_ch),
            nn.ReLU(),
            nn.Conv2d(out_ch, out_ch, 3, padding=1),
            nn.BatchNorm2d(out_ch),
            nn.ReLU()
        )
        self.mlp = nn.Sequential(
            nn.Linear(time_embed_dim, in_ch),
            nn.ReLU(),
            nn.Linear(in_ch, in_ch)
        )

    def forward(self, x, v):
        N, C, _, _ = x.shape
        v = self.mlp(v)
        v = v.view(N, C, 1, 1)
        y = self.convs(x + v)
        return y

in_ch = 1
time_embed_dim=100
timesteps = torch.tensor([1])

down1 = ConvBlock(in_ch, 64, time_embed_dim)
down2 = ConvBlock(64,   128, time_embed_dim)
down3 = ConvBlock(128,  256, time_embed_dim)

bot1 = ConvBlock(256, 512, time_embed_dim)

up3 = ConvBlock(128 + 512, 256, time_embed_dim)
up2 = ConvBlock(128 + 256, 128, time_embed_dim)
up1 = ConvBlock(128 + 64 , 64 , time_embed_dim)
out = nn.Conv2d(64, in_ch, 1)

maxpool = nn.MaxPool2d(2)
upsample = nn.Upsample(scale_factor=2, mode='bilinear')

In [66]:
def _pos_encoding(time_idx, output_dim, device='cpu'):
    t, D = time_idx, output_dim
    v = torch.zeros(D, device=device)

    i = torch.arange(0, D, device=device)
    div_term = torch.exp(i / D * math.log(10000))

    v[0::2] = torch.sin(t / div_term[0::2])
    v[1::2] = torch.cos(t / div_term[1::2])
    return v

def pos_encoding(timesteps, output_dim, device='cpu'):
    batch_size = len(timesteps)
    device = timesteps.device
    v = torch.zeros(batch_size, output_dim, device=device)
    for i in range(batch_size):
        v[i] = _pos_encoding(timesteps[i], output_dim, device)
    return v

x = torch.randn(1, 1, 28, 28)

v = pos_encoding(timesteps, time_embed_dim, x.device)

x1 = down1(x, v)
x = maxpool(x1)

x2 = down2(x, v)
x = maxpool(x2)
print(x.shape)
x3 = down3(x, v)
x = maxpool(x3)

print(x.shape)

x = upsample(x)
x = torch.cat([x, x3], dim=1)
x = up3(x, v)

x = upsample(x)
x = torch.cat([x, x2], dim=1)
x = up2(x, v)

x = upsample(x)
x = torch.cat([x, x1], dim=1)
x = up1(x, v)

x = out(x)

torch.Size([1, 128, 7, 7])
torch.Size([1, 256, 3, 3])


RuntimeError: Sizes of tensors must match except in dimension 1. Expected size 6 but got size 7 for tensor number 1 in the list.

単純に、層を増や「そう」としたが、maxpoolingで形状が半分になるため難しい（奇数の場合は切り捨てされる模様）

strideを変更すればできなくはない？

upsampleをdecomvolutionに変更させる

In [140]:
def _pos_encoding(time_idx, output_dim, device='cpu'):
    t, D = time_idx, output_dim
    v = torch.zeros(D, device=device)

    i = torch.arange(0, D, device=device)
    div_term = torch.exp(i / D * math.log(10000))

    v[0::2] = torch.sin(t / div_term[0::2])
    v[1::2] = torch.cos(t / div_term[1::2])
    return v

def pos_encoding(timesteps, output_dim, device='cpu'):
    batch_size = len(timesteps)
    device = timesteps.device
    v = torch.zeros(batch_size, output_dim, device=device)
    for i in range(batch_size):
        v[i] = _pos_encoding(timesteps[i], output_dim, device)
    return v

class ConvBlock(nn.Module):
    def __init__(self, in_ch, out_ch, time_embed_dim):
        super().__init__()
        self.convs = nn.Sequential(
            nn.Conv2d(in_ch, out_ch, 3, padding=1),
            nn.BatchNorm2d(out_ch),
            nn.ReLU(),
            nn.Conv2d(out_ch, out_ch, 3, padding=1),
            nn.BatchNorm2d(out_ch),
            nn.ReLU()
        )
        self.mlp = nn.Sequential(
            nn.Linear(time_embed_dim, in_ch),
            nn.ReLU(),
            nn.Linear(in_ch, in_ch)
        )

    def forward(self, x, v):
        N, C, _, _ = x.shape
        v = self.mlp(v)
        v = v.view(N, C, 1, 1)
        y = self.convs(x + v)
        return y

class DeConvBlock(nn.Module):
    def __init__(self, in_ch, out_ch, time_embed_dim):
        super().__init__()
        self.deconvs = nn.Sequential(
            nn.ConvTranspose2d(in_ch, out_ch, 6, padding=1),
            nn.BatchNorm2d(out_ch),
            nn.ReLU(),
            nn.ConvTranspose2d(out_ch, out_ch, 6, padding=1),
            nn.BatchNorm2d(out_ch),
            nn.ReLU()
        )
        self.mlp = nn.Sequential(
            nn.Linear(time_embed_dim, in_ch),
            nn.ReLU(),
            nn.Linear(in_ch, in_ch)
        )
    def forward(self, x, v):
        N, C, _, _, = x.shape
        # v = self.mlp(v)
        # v = v.view(N, C, 1, 1)
        y = self.deconvs(x)
        return y
    
class UNet(nn.Module):
    def __init__(self, in_ch=1, time_embed_dim=100):
        super().__init__()
        self.time_embed_dim = time_embed_dim

        self.down1 = ConvBlock(in_ch, 64, time_embed_dim)
        self.down2 = ConvBlock(64, 128, time_embed_dim)
        self.bot1 = ConvBlock(128, 256, time_embed_dim)
        self.up2 = ConvBlock(128 + 256, 128, time_embed_dim)
        self.up1 = ConvBlock(128 + 64, 64, time_embed_dim)
        self.out = nn.Conv2d(64, in_ch, 1)

        self.maxpool = nn.MaxPool2d(2)
        self.upsample = nn.Upsample(scale_factor=2, mode='bilinear')

    def forward(self, x, timesteps):
        v = pos_encoding(timesteps, self.time_embed_dim, x.device)

        x1 = self.down1(x, v)
        x = self.maxpool(x1)
        x2 = self.down2(x, v)
        x = self.maxpool(x2)

        x = self.bot1(x, v)

        x = self.upsample(x)
        x = torch.cat([x, x2], dim=1)
        x = self.up2(x, v)
        x = self.upsample(x)
        x = torch.cat([x, x1], dim=1)
        x = self.up1(x, v)
        x = self.out(x)
        return x


In [141]:
x = torch.randn(1, 1, 28, 28)

deconv = DeConvBlock(1, 64, 10)
v = pos_encoding(timesteps, time_embed_dim, x.device)
y = deconv(x,v)

In [142]:
x = torch.randn(1, 1, 28, 28)

v = pos_encoding(timesteps, time_embed_dim, x.device)

x1 = down1(x, v)
x = maxpool(x1)

x2 = down2(x, v)
x = maxpool(x2)

print(x.shape)

deconv1 = DeConvBlock(128, 64, time_embed_dim)

x = deconv1(x, v)

print(x.shape)

deconv2 = DeConvBlock(64, 32, time_embed_dim)

x = deconv2(x, v)

print(x.shape)

torch.Size([1, 128, 7, 7])
torch.Size([1, 64, 13, 13])
torch.Size([1, 32, 19, 19])
